In [2]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("movie_ratings").getOrCreate()

Load the dataset into a PySpark DataFrame.

In [4]:
links_df = spark.read.csv("ml-latest-small/links.csv",header=True)
movies_df = spark.read.csv("ml-latest-small/movies.csv",header=True)
ratings_df = spark.read.csv("ml-latest-small/ratings.csv",header=True)
tags_df = spark.read.csv("ml-latest-small/tags.csv",header=True)

Calculate the total number of ratings in the dataset.

In [5]:
total_rating = ratings_df.count()
print("the total ratings is :",total_rating)

the total ratings is : 100836


Calculate the average rating for each movie.

In [6]:
avg_rating = ratings_df.groupBy('movieId').agg({'rating':'avg'}).orderBy('movieId')

Find the top 10 movies with the highest average rating.

In [8]:
top_10_avg_rating = ratings_df.orderBy(desc("rating")).limit(10)

In [9]:
top_10_avg_rating.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    362|   5.0|964982588|
|     1|    231|   5.0|964981179|
|     1|     50|   5.0|964982931|
|     1|    260|   5.0|964981680|
|     1|    151|   5.0|964984041|
|     1|    333|   5.0|964981179|
|     1|     47|   5.0|964983815|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+



Calculate the average rating for each user.

In [10]:
avg_rating_per_user = ratings_df.groupBy('userId').agg({'rating':'avg'})

In [11]:
avg_rating_per_user.show()

+------+------------------+
|userId|       avg(rating)|
+------+------------------+
|   296| 4.166666666666667|
|   467| 3.409090909090909|
|   125| 3.859722222222222|
|   451|3.7941176470588234|
|     7|3.2302631578947367|
|    51|3.7757660167130918|
|   124|              3.99|
|   447| 3.871794871794872|
|   591|3.2777777777777777|
|   307|2.6656410256410257|
|   475| 4.409677419354839|
|   574|3.9565217391304346|
|   169|  4.24907063197026|
|   205|3.8703703703703702|
|   334| 3.418831168831169|
|   544| 4.363636363636363|
|   577|3.5652173913043477|
|   581|             4.375|
|   272| 3.629032258064516|
|   442|             1.275|
+------+------------------+
only showing top 20 rows



Find the user with the highest average rating.

In [12]:
user_with_highest_avg_rating = avg_rating_per_user.orderBy(desc('avg(rating)')).limit(1)

In [13]:
user_with_highest_avg_rating.show()

+------+-----------+
|userId|avg(rating)|
+------+-----------+
|    53|        5.0|
+------+-----------+



Calculate the number of ratings given by each user.

In [194]:
no_of_rating_by_each_user = ratings_df.groupBy('userId').agg({'rating':'count'})

In [15]:
no_of_rating_by_each_user.show()

+------+-------------+
|userId|count(rating)|
+------+-------------+
|   296|           27|
|   467|           22|
|   125|          360|
|   451|           34|
|     7|          152|
|    51|          359|
|   124|           50|
|   447|           78|
|   591|           54|
|   307|          975|
|   475|          155|
|   574|           23|
|   169|          269|
|   205|           27|
|   334|          154|
|   544|           22|
|   577|          161|
|   581|           40|
|   272|           31|
|   442|           20|
+------+-------------+
only showing top 20 rows



Find the top 5 users who have given the most ratings.

In [16]:
top_5_user_given_most_rating = no_of_rating_by_each_user.orderBy(desc('count(rating)')).limit(5)

In [17]:
top_5_user_given_most_rating.show()

+------+-------------+
|userId|count(rating)|
+------+-------------+
|   414|         2698|
|   599|         2478|
|   474|         2108|
|   448|         1864|
|   274|         1346|
+------+-------------+



Find the genre with the highest average rating.

In [19]:
genre_with_highest_avg_rating = avg_rating.join(movies_df,'movieId')

In [20]:
genre_with_highest_avg_rating = genre_with_highest_avg_rating.select('avg(rating)','genres').orderBy(desc('avg(rating)'))

In [21]:
genre_with_highest_avg_rating.first()

Row(avg(rating)=5.0, genres='Comedy')

Calculate the average rating for each genre

In [22]:
genre_with_ratings = avg_rating.join(movies_df,'movieId').select('avg(rating)','genres')

In [23]:
genre_with_ratings.show()

+------------------+--------------------+
|       avg(rating)|              genres|
+------------------+--------------------+
| 4.197068403908795|Comedy|Crime|Dram...|
| 3.984126984126984|           Drama|War|
|3.9107142857142856|Drama|Sci-Fi|Thri...|
|3.4761904761904763|Comedy|Drama|Romance|
|          3.546875|Action|Adventure|...|
|2.6666666666666665|Comedy|Fantasy|Mu...|
|               2.5|Adventure|Comedy|...|
|3.2444444444444445|Adventure|Animati...|
|               3.1|  Adventure|Thriller|
|             3.975|      Drama|Thriller|
|             3.625|Action|Adventure|...|
|3.6315789473684212|        Comedy|Drama|
|2.4642857142857144|       Comedy|Sci-Fi|
|3.3333333333333335|      Comedy|Romance|
|              3.75|Comedy|Musical|Ro...|
|               4.0|Adventure|Childre...|
|             3.625|Horror|Sci-Fi|Thr...|
|              3.75|Drama|Horror|Myst...|
|               4.5|               Drama|
|               4.5|         Documentary|
+------------------+--------------

Determine the top 5 movies with the highest rating count (number of ratings).

In [24]:
highest_rating_count = ratings_df.groupBy('movieId').agg({'rating':'count'})

In [25]:
highest_rating_count=highest_rating_count.orderBy(desc('count(rating)')).join(movies_df,'movieId').select('title','count(rating)').limit(5)

In [26]:
highest_rating_count.show()

+--------------------+-------------+
|               title|count(rating)|
+--------------------+-------------+
| Pulp Fiction (1994)|          307|
|      Platoon (1986)|           63|
|   Ex Machina (2015)|           28|
|Fast Times at Rid...|           42|
|Captain America: ...|           32|
+--------------------+-------------+



Calculate the average rating for movies released in each year.

In [184]:
movies_with_year = movies_df.withColumn('year',regexp_extract(movies_df.title,'\((\d{4})\)',1))

In [185]:
avg_rating_per_year = movies_with_year.join(ratings_df,'movieId')

In [186]:
avg_rating_per_year = avg_rating_per_year.groupBy('year').agg({'rating':'avg'})

In [187]:
avg_rating_per_year.show()

+----+------------------+
|year|       avg(rating)|
+----+------------------+
|1953|3.6470588235294117|
|1903|               2.5|
|1957|  4.03953488372093|
|1987|3.4883116883116885|
|1956|3.6826086956521737|
|2016| 3.387261146496815|
|1936|3.5283018867924527|
|2012|3.5126262626262625|
|1958|3.8424242424242423|
|1943|             3.625|
|1915|               2.0|
|1972| 3.939890710382514|
|1931|3.6052631578947367|
|1938| 3.767857142857143|
|1988|3.4181173436492585|
|1926|               4.0|
|2017|3.5780911062906724|
|1932|3.3333333333333335|
|1977| 3.810405643738977|
|1971| 3.841463414634146|
+----+------------------+
only showing top 20 rows



Find the user(s) who have rated the largest number of movies across different genres

In [31]:
rating_with_genre = ratings_df.join(movies_df,'movieId')
rating_with_genre.show()

+-------+------+------+---------+--------------------+--------------------+
|movieId|userId|rating|timestamp|               title|              genres|
+-------+------+------+---------+--------------------+--------------------+
|      1|     1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|
|      3|     1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|
|      6|     1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|
|     47|     1|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   5.0|964982931|Usual Suspects, T...|Crime|Mystery|Thr...|
|     70|     1|   3.0|964982400|From Dusk Till Da...|Action|Comedy|Hor...|
|    101|     1|   5.0|964980868|Bottle Rocket (1996)|Adventure|Comedy|...|
|    110|     1|   4.0|964982176|   Braveheart (1995)|    Action|Drama|War|
|    151|     1|   5.0|964984041|      Rob Roy (1995)|Action|Drama|Roma...|
|    157|     1|   5.0|964984100|Canadian Bacon (1...|          Comedy|War|
|    163|   

In [32]:
rating_with_genre.groupBy('userId','genres').count().orderBy(desc('count')).show()

+------+--------------------+-----+
|userId|              genres|count|
+------+--------------------+-----+
|   474|               Drama|  319|
|   448|              Comedy|  271|
|   414|               Drama|  257|
|   414|              Comedy|  238|
|   599|              Comedy|  226|
|   599|               Drama|  186|
|   111|              Comedy|  173|
|   606|               Drama|  167|
|   603|               Drama|  153|
|   474|              Comedy|  152|
|   298|              Comedy|  150|
|   307|              Comedy|  142|
|   414|        Comedy|Drama|  137|
|   274|              Comedy|  130|
|   474|       Drama|Romance|  124|
|   414|      Comedy|Romance|  121|
|    89|              Comedy|  105|
|   414|Comedy|Drama|Romance|  105|
|   414|       Drama|Romance|  102|
|   606|       Drama|Romance|   99|
+------+--------------------+-----+
only showing top 20 rows



Calculate the average rating for each genre, considering only movies with more than 100 ratings

In [76]:
movies_with_more_than_hundred_rating = ratings_df.join(movies_df,'movieId')
# .groupBy('genres').agg({'rating':'avg'})

In [82]:
movies_with_more_than_hundred_rating=movies_with_more_than_hundred_rating.groupBy('genres').agg({'rating':'avg'})

In [87]:
avg_ratings_more_than_100_rating = ratings_df.groupBy('movieId').agg({'rating':'count'}).filter('count(rating)>100').join(movies_df,'movieId').join(movies_with_more_than_hundred_rating,'genres')
# .groupBy('genres').agg({'rating':'avg'})

In [88]:
avg_ratings_more_than_100_rating.show()

+--------------------+-------+-------------+--------------------+------------------+
|              genres|movieId|count(rating)|               title|       avg(rating)|
+--------------------+-------+-------------+--------------------+------------------+
|Action|Adventure|...|   7153|          185|Lord of the Rings...|3.9191176470588234|
|Action|Adventure|...|   3578|          170|    Gladiator (2000)|3.7463465553235906|
|   Adventure|Fantasy|   5952|          188|Lord of the Rings...| 3.872431506849315|
|   Adventure|Fantasy|   4993|          198|Lord of the Rings...| 3.872431506849315|
|   Adventure|Fantasy|   5816|          102|Harry Potter and ...| 3.872431506849315|
|Crime|Drama|Thriller|  48516|          107|Departed, The (2006)|  3.73503127792672|
|Crime|Drama|Thriller|    111|          104|  Taxi Driver (1976)|  3.73503127792672|
|Mystery|Sci-Fi|Th...|     32|          177|Twelve Monkeys (a...|3.8725868725868726|
|Adventure|Drama|IMAX|    150|          201|    Apollo 13 (1995)|

Identify the movie(s) that received the highest and lowest ratings from users who have rated at least 100 movies.

In [102]:
user_with_100_ratings = ratings_df.groupBy('userId').agg({"rating":"count"}).filter("count(rating)>100")

In [103]:
user_with_100_ratings.show()

+------+-------------+
|userId|count(rating)|
+------+-------------+
|   125|          360|
|     7|          152|
|    51|          359|
|   307|          975|
|   475|          155|
|   169|          269|
|   334|          154|
|   577|          161|
|   462|          455|
|    15|          135|
|   232|          862|
|   234|          202|
|   282|          237|
|   448|         1864|
|   483|          728|
|   132|          347|
|   317|          189|
|   200|          334|
|   428|          300|
|   495|          265|
+------+-------------+
only showing top 20 rows



In [110]:
movie_ratings = user_with_100_ratings.join(ratings_df,'userId').orderBy('rating')

In [109]:
# user_with_100_ratings.join(ratings_df,'userId').orderBy('rating').tail(1)

[Row(userId='610', count(rating)=1302, movieId='168252', rating='5.0', timestamp='1493846352')]

In [112]:
print("lowest rated movie by user with atleast 100 ratings : ",movie_ratings.first())
print("highest rated movie by user with atleast 100 ratings : ",movie_ratings.tail(1))


lowest rated movie by user with atleast 100 ratings :  Row(userId='7', count(rating)=152, movieId='1784', rating='0.5', timestamp='1106635416')
highest rated movie by user with atleast 100 ratings :  [Row(userId='610', count(rating)=1302, movieId='168252', rating='5.0', timestamp='1493846352')]


Calculate the average rating for each genre, excluding movies released in the last 5 years.

In [188]:
avg_rating_per_genre_until_last_5_yr = movies_with_year.filter("year<=2018")

In [189]:
avg_ratings = ratings_df.groupBy('movieId').agg(count('rating').alias("avg_rating"))

In [190]:
avg_rating_per_genre_until_last_5_yr=avg_ratings.join(avg_rating_per_genre_until_last_5_yr,"movieId")

In [191]:
avg_rating_per_genre_until_last_5_yr.show()

+-------+----------+--------------------+--------------------+----+
|movieId|avg_rating|               title|              genres|year|
+-------+----------+--------------------+--------------------+----+
|    296|       307| Pulp Fiction (1994)|Comedy|Crime|Dram...|1994|
|   1090|        63|      Platoon (1986)|           Drama|War|1986|
| 115713|        28|   Ex Machina (2015)|Drama|Sci-Fi|Thri...|2015|
|   3210|        42|Fast Times at Rid...|Comedy|Drama|Romance|1982|
|  88140|        32|Captain America: ...|Action|Adventure|...|2011|
|    829|         9|Joe's Apartment (...|Comedy|Fantasy|Mu...|1996|
|   2088|        18|       Popeye (1980)|Adventure|Comedy|...|1980|
|   2294|        45|         Antz (1998)|Adventure|Animati...|1998|
|   4821|         5|     Joy Ride (2001)|  Adventure|Thriller|2001|
|  48738|        20|Last King of Scot...|      Drama|Thriller|2006|
|   3959|         8|Time Machine, The...|Action|Adventure|...|1960|
|  89864|        19|        50/50 (2011)|       